In [207]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [518]:
import os
import pandas as pd
import numpy as np
import sys

from tkinter import *
import warnings
import tkinter as tk

In [519]:
local_path = 'C:/Users/USER/Desktop/CIG/data/'

In [520]:
# Excel path, name, sheet, row,column configuration

excel_date = '220723'
in_xl_name = 'input_excel_data' # + excel_date
out_xl_name = 'output_excel_raw_data'# + excel_date

in_xl_path = local_path + in_xl_name + '.xlsx'
sheet_name = '리스트'
out_xl_path = local_path + out_xl_name + '.xls'

In [521]:
input_df = pd.read_excel(in_xl_path, sheet_name= sheet_name)

In [522]:
custom_col = ['H.BL NO',
              'M.BL NO',
              'SHIPPER',
              'CNEE',
              'ACID',
              'MODEL',
              'YR',
              'CHASSINO.',
              'WEIGHT',
              'CBM']

In [523]:
input_df = input_df[custom_col]

In [524]:
skip_row_nums =2 
output_df = pd.read_excel(out_xl_path, skiprows=skip_row_nums)
output_df = output_df.dropna(how='all')

In [525]:
h_bl_no_list = list(input_df['H.BL NO'].drop_duplicates())

In [526]:
def _custom_round(number, n=1): # embedded python round func could cause rounding issue at point 5
    n = n+2
    pow_ten = pow(10, n)
    
    number = number * pow_ten
    
    if number % 10 > 4:
        number += (10 - number % 10)
    
    number = number / pow_ten
    return float(str(number)[:n+2]) 

In [527]:
def _check_total_weight(in_df,out_df): # floating round : 1
    f_num = 1
    in_tot_weight = _custom_round(in_df['WEIGHT'].sum(),f_num)
    out_tot_weight = _custom_round(out_df['G.W/T'].sum(),f_num)
    return in_tot_weight,out_tot_weight

def _check_total_CBM(in_df,out_df): # floating round : 3
    f_num = 3
    in_tot_CBM = _custom_round(in_df['CBM'].sum(),f_num)
    out_tot_CBM = _custom_round(out_df['CBM'].sum(),f_num)
    return in_tot_CBM,out_tot_CBM

In [528]:
def _find_location(out_xl_df,hbl,out_data,category):
    row_index = 4
    if category == 'weight':
        match_df = out_xl_df.copy().loc[((out_xl_df['HOUSE NO'] == hbl) & (out_xl_df['G.W/T'] == out_data)),:]
    elif category == 'cbm':
        match_df = out_xl_df.copy().loc[((out_xl_df['HOUSE NO'] == hbl) & (out_xl_df['CBM'] == out_data)),:]
    fail_index = match_df.index
    fail_row = list(row_index + fail_index)
    
    return fail_row

In [529]:
def _gen_error_win():
    window = Tk()
    window.title("Error Message!")
    return window

In [531]:
from cig_script.config.config_env import Config

def _out_total_weight_or_cbm_error_default_labeling(window):
    #    Error message 
    lb_err_msg1 = Label(window, text="This error is caused if total weight or cbm mismatching in output file was found!")

    lb_err_msg1.place(x=10, y=10)
    
    lb_err_msg2 = Label(window, text="You should check below location of empty data in output file!")
    lb_err_msg2.place(x=10, y=30)
    
    #    Current working directory path
    working_xl_path = Config().out_xl_path 

    lb_err_working_dir = Label(window, text="Working Directory Path : ")
    lb_err_working_dir.place(x=10, y=70)
    
    ety_working_path = tk.Entry(fg="gray19", bg="snow", width=60)
    ety_working_path.place(x=10,y=90)
    ety_working_path.insert(0,working_xl_path)

In [603]:
def _call_error_message_window(window,*err_dict):
    # label,entry location config
    default_y_row = 90
    lb_y_step_size = 30
    lb_x_row = 10
    ety_y_step_size = 60
    ety_x_row = 10
    
    dict_num = len(err_dict)
    for num in range(dict_num):
        curr_err_dict = err_dict[num] 
        for err_cnt in curr_err_dict:
            empty_rows,empty_col = curr_err_dict[err_cnt]

            #    N th Error Row Location
            lb_y_row = default_y_row + lb_y_step_size

            lb_err_empty_row = Label(window, text= "Error Row Location : ")
            lb_err_empty_row.place(x=lb_x_row,y=lb_y_row)

            ety_y_row = default_y_row + ety_y_step_size 

            ety_empty_row = tk.Entry(fg="gray19", bg="snow", width=30)
            ety_empty_row.place(x=ety_x_row,y=ety_y_row)
            ety_empty_row.insert(0,empty_rows)

            #  N th  Error Column Location
            lb_err_empty_col = Label(window, text= "Error Column Location : ")
            lb_err_empty_col.place(x=250, y=lb_y_row)
            ety_empty_col = tk.Entry(fg="gray19", bg="snow", width=30)
            ety_empty_col.place(x=250,y=ety_y_row)
            ety_empty_col.insert(0,empty_col)

            default_y_row = ety_y_row
    
    window.geometry('600x600')
    
    window.mainloop()

In [629]:
def _find_fail_loc_out(out_xl_df,fail_w_data,fail_c_data,fail_w_info,fail_c_info):
    in_w_weight,in_c_weight = 0,0
    out_w_weight,out_c_weight = 0,0
    find_w_loc_dict,find_c_loc_dict = {},{}
    fail_w_row,fail_c_row = [],[]
    
    w_cat = fail_w_info['category']
    c_cat = fail_c_info['category']
    
    cat_num = 0
    for hbl in fail_w_data:
        
        w_cnt = fail_w_info[hbl]
        in_w_weight,out_w_weight = fail_w_data[hbl]
        fail_w_row = fail_w_row + (_find_location(out_xl_df,hbl,out_w_weight,w_cat))
        
    cat_num = cat_num + 1
    find_w_loc_dict.update({cat_num:(fail_w_row,w_cat)})
        
    for hbl in fail_c_data:
        
        c_cnt = fail_c_info[hbl]
        in_c_weight,out_c_weight = fail_c_data[hbl]
        fail_c_row = fail_c_row + (_find_location(out_xl_df,hbl,out_c_weight,c_cat))
    
    cat_num = cat_num + 1
    find_c_loc_dict.update({cat_num:(fail_c_row,c_cat)})
    
    window = _gen_error_win()
    _out_total_weight_or_cbm_error_default_labeling(window)
    _call_error_message_window(window,find_w_loc_dict,find_c_loc_dict)

In [630]:
def call_fail_message(out_xl_df,fail_w_data,fail_c_data,fail_w_info,fail_c_info):
    _find_fail_loc_out(out_xl_df,fail_w_data,fail_c_data,fail_w_info,fail_c_info)

In [631]:
def _check_comparison_in_out_df(in_xl_df,out_xl_df):
    pass_w_data,pass_c_data = {},{}
    fail_w_data,fail_c_data = {},{}
    empty_w_data,empty_c_data = {},{}
    fail_w_info,fail_c_info = {},{}

    pass_w_cnt,pass_c_cnt = 0,0
    fail_w_cnt,fail_c_cnt = 0,0
    empty_w_cnt,empty_c_cnt = 0,0

    h_bl_no_list = list(in_xl_df['H.BL NO'].drop_duplicates())
    
    for li in h_bl_no_list:

        input_list_match_df = in_xl_df.copy().loc[(in_xl_df['H.BL NO'] == li),:]
        output_list_match_df = out_xl_df.copy().loc[(out_xl_df['HOUSE NO']==li),:]

        # Total Weight
        in_tot_weight, out_tot_weight =  _check_total_weight(input_list_match_df,output_list_match_df)
    
        if in_tot_weight == out_tot_weight:
            pass_w_cnt = pass_w_cnt + 1
            pass_w_data.update({li:(in_tot_weight,out_tot_weight)})
        elif (in_tot_weight != 0) and (out_tot_weight != 0):
            category = 'weight'
            fail_w_cnt = fail_w_cnt + 1
            fail_w_data.update({li:(in_tot_weight,out_tot_weight)})
            fail_w_info.update({li:fail_w_cnt,'category':category})
        elif (in_tot_weight != 0) or (out_tot_weight != 0):
            empty_w_cnt = empty_w_cnt + 1
            empty_w_data.update({li:(in_tot_weight,out_tot_weight)})
        else:
            print("please check whether input or output weight data is valid")
        
        # Total CBM
        in_tot_CBM, out_tot_CBM =  _check_total_CBM(input_list_match_df,output_list_match_df)
    
        if in_tot_CBM == out_tot_CBM:
            pass_c_cnt = pass_c_cnt + 1
            pass_c_data.update({li:(in_tot_CBM,out_tot_CBM)})
        elif (in_tot_CBM != 0) and (out_tot_CBM != 0):
            category = 'cbm'
            fail_c_cnt = fail_c_cnt + 1
            fail_c_data.update({li:(in_tot_CBM,out_tot_CBM)})
            fail_c_info.update({li:fail_c_cnt,'category':category})
        elif (in_tot_CBM != 0) or (out_tot_CBM != 0):
            empty_c_cnt = empty_c_cnt + 1
            empty_c_data.update({li:(in_tot_CBM,out_tot_CBM)})
        else:
            print("please check whether input or output CBM data is valid")

    total_w_cnt = pass_w_cnt + fail_w_cnt + empty_w_cnt
    total_c_cnt = pass_c_cnt + fail_c_cnt + empty_c_cnt
    # print(pass_w_data)
    # print(pass_w_cnt)
#     print(fail_w_data)
    # print(fail_w_cnt)
    # print(empty_w_data)
    # print(empty_w_cnt)

    # print(pass_c_data)
    # print(pass_c_cnt)
#     print(fail_c_data)
    # print(fail_c_cnt)
    # print(empty_c_data)
    # print(empty_c_cnt)
#     fail_dict = fail_w_data.update(fail_c_data) 
#     fail_cnt = (fail_w_cnt,fail_c_cnt)
#     print(fail_cnt)
    call_fail_message(out_xl_df,fail_w_data,fail_c_data,fail_w_info,fail_c_info)

In [632]:
def check_comparison_in_out_df(in_xl_df, out_xl_df):
    _check_comparison_in_out_df(in_xl_df,out_xl_df)

In [633]:
check_comparison_in_out_df(input_df, output_df)